In [16]:
ls Z:\Shared243\storal\2024-01-23_mc191

 Volume in drive Z is SYSTEM
 Volume Serial Number is ECF6-6273

 Directory of Z:\Shared243\storal\2024-01-23_mc191

15/02/2024  14:52    <DIR>          .
20/02/2024  17:39    <DIR>          ..
07/02/2024  10:38         3,484,463 488PSF.ome.tif
07/02/2024  16:41                73 488PSF_sigma.csv
23/01/2024  14:03         7,841,672 PSFs.sld
23/01/2024  17:48    46,071,658,154 slide1.sld
07/02/2024  15:42    71,967,135,033 slide1_deskewed.sld
05/02/2024  20:42     2,248,488,799 slide1_deskewed_capture1_cropped_488.ome.tif
14/02/2024  13:41     4,085,934,729 slide1_deskewed_capture1_cropped_merge_deconvolved.ims
14/02/2024  13:20     4,354,120,629 slide1_deskewed_capture1_cropped_merge_deconvolved.tif
05/02/2024  20:36     1,749,500,785 slide1_deskewed_capture4_cropped_488.ome.tif
14/02/2024  13:42     3,133,630,327 slide1_deskewed_capture4_cropped_merge_deconvolved.ims
14/02/2024  13:18     3,402,338,332 slide1_deskewed_capture4_cropped_merge_deconvolved.tif
05/02/2024  20:44     1,980,

In [17]:
psf1 = 'Z:/Shared243/storal/2024-01-17_mc191/488PSF_sigma.csv'
psf2 = 'Z:/Shared243/storal/2024-01-18_mc191/488PSF_sigma.csv'
psf3 = 'Z:/Shared243/storal/2024-01-23_mc191/488PSF_sigma.csv'



In [18]:
import numpy as np

def average_csv_files(file_paths):
    # Read data from each CSV file into NumPy arrays
    arrays = [np.genfromtxt(file_path, delimiter=',') for file_path in file_paths]

    # Compute the element-wise average of the arrays
    average_array = np.mean(arrays, axis=0)

    return average_array

# Paths to the CSV files
csv_files = [psf1, psf2, psf3]

# Compute the average of the CSV files
average_data = average_csv_files(csv_files)

# Optionally, save the average array to a new CSV file
np.savetxt('average.csv', average_data, delimiter=',', fmt='%f')


In [3]:
import numpy as np
from skimage.registration import phase_cross_correlation
from scipy.ndimage import fourier_shift
import tifffile

def load_psf(paths):
    """Load PSF TIF files from the given paths."""
    psfs = [tifffile.imread(path) for path in paths]
    return psfs

def align_psfs(psfs):
    """Align PSFs using phase cross-correlation."""
    reference_psf = psfs[0]
    aligned_psfs = [reference_psf]
    
    for psf in psfs[1:]:
        shift, _, _ = phase_cross_correlation(reference_psf, psf, upsample_factor=10)
        shifted_psf = np.real(np.fft.ifftn(fourier_shift(np.fft.fftn(psf), shift)))
        aligned_psfs.append(shifted_psf)
        
    return aligned_psfs

def average_psfs(psfs):
    """Average the aligned PSFs."""
    return np.mean(psfs, axis=0)

def save_psf(psf, output_path):
    """Save the averaged PSF to a new TIF file."""
    tifffile.imwrite(output_path, psf.astype(np.float32))

def main(psf_paths, output_path):
    psfs = load_psf(psf_paths)
    aligned_psfs = align_psfs(psfs)
    averaged_psf = average_psfs(aligned_psfs)
    save_psf(averaged_psf, output_path)

# Example usage
psf_paths = ['PSFs/2024_01_23_488PSF.ome.tif', 'PSFs/2024_01_25_488PSF.ome.tif', 'PSFs/2024_01_26_488PSF.ome.tif', 'PSFs/2024_01_30_488PSF.ome.tif']  # Update with your actual file paths
output_path = 'PSFs/averaged_psf.tif'
main(psf_paths, output_path)
